In [3]:
import keras
from keras import models
from keras import layers
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_csv('data.csv')
class_mapping = {label:idx for idx,label in
               enumerate(np.unique(data[ 'diagnosis' ] )) }
x = data.iloc[ : , 2: ]  
y = data.iloc [ : , 1].values
for i in range(len(y)):
    y[i] = class_mapping[y[i]]
x_train, x_test, y_train, y_test = train_test_split(x, y , test_size=0.1, random_state=0)

x_train = np.asarray(x_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

In [4]:
class NN():
    def __init__(self, BatchSize, ActFunc, Optimizer, Loss, NeuronList):
        self.ActFunc = ActFunc
        self.Optimizer = Optimizer
        self.BatchSize = BatchSize
        self.LayerNumber = 2
        self.NeuronList = NeuronList
        self.CrossValidCount = 3
        self.Loss = Loss
        self.Chromosome = [BatchSize, ActFunc, Optimizer, Loss, NeuronList]

    def ModelFit(self, train_data, train_labels):
        network = models.Sequential()
        network.add(layers.Dense(self.NeuronList[0], activation=self.ActFunc, input_shape=(30, )))
        for i in range(1, self.LayerNumber):
            network.add(layers.Dense(self.NeuronList[i], activation=self.ActFunc))
        network.add(layers.Dense(1, activation='sigmoid'))

        network.compile(optimizer=self.Optimizer, loss = self.Loss,
                                            metrics=['acc'])
        self.history = [0 for i in range(self.CrossValidCount)]
        for i in range(self.CrossValidCount):
            self.history[i] = network.fit(train_data, train_labels, epochs=50, batch_size=self.BatchSize, verbose=1, validation_split = 0.3)
        self.EvalAcc()
        
    def EvalAcc(self):
        self.Acc = 0
        for i in range(self.CrossValidCount):
            self.Acc += max(self.history[i].history['val_acc'])
        self.Acc /= self.CrossValidCount
    
    def GetAcc(self):
        return self.Acc

In [5]:
Network = NN(BatchSize=10, ActFunc='sigmoid', Optimizer='rmsprop', Loss='MSE', NeuronList=[20, 20])
Network.ModelFit(x_train, y_train)

Epoch 1/50
36/36 [==============================] - 0s 5ms/step - loss: 0.2691 - acc: 0.5000 - val_loss: 0.2287 - val_acc: 0.8117
Epoch 2/50
36/36 [==============================] - 0s 1ms/step - loss: 0.2152 - acc: 0.6899 - val_loss: 0.2117 - val_acc: 0.6364
Epoch 3/50
36/36 [==============================] - 0s 1ms/step - loss: 0.2011 - acc: 0.6816 - val_loss: 0.1981 - val_acc: 0.6818
Epoch 4/50
36/36 [==============================] - 0s 1ms/step - loss: 0.1837 - acc: 0.7570 - val_loss: 0.1830 - val_acc: 0.7273
Epoch 5/50
36/36 [==============================] - 0s 1ms/step - loss: 0.1631 - acc: 0.8352 - val_loss: 0.1629 - val_acc: 0.8182
Epoch 6/50
36/36 [==============================] - 0s 1ms/step - loss: 0.1466 - acc: 0.8827 - val_loss: 0.1484 - val_acc: 0.8377
Epoch 7/50
36/36 [==============================] - 0s 1ms/step - loss: 0.1308 - acc: 0.9050 - val_loss: 0.1387 - val_acc: 0.8831
Epoch 8/50
36/36 [==============================] - 0s 1ms/step - loss: 0.1229 - acc: 0.89

36/36 [==============================] - 0s 2ms/step - loss: 0.0503 - acc: 0.9385 - val_loss: 0.0798 - val_acc: 0.9026
Epoch 14/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0598 - acc: 0.9190 - val_loss: 0.0669 - val_acc: 0.9156
Epoch 15/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0546 - acc: 0.9246 - val_loss: 0.0692 - val_acc: 0.9091
Epoch 16/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0539 - acc: 0.9302 - val_loss: 0.0774 - val_acc: 0.9026
Epoch 17/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0597 - acc: 0.9134 - val_loss: 0.0657 - val_acc: 0.9091
Epoch 18/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0548 - acc: 0.9162 - val_loss: 0.0674 - val_acc: 0.8831
Epoch 19/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0600 - acc: 0.9106 - val_loss: 0.0717 - val_acc: 0.9221
Epoch 20/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0624 - acc: 0.9050 -

36/36 [==============================] - 0s 2ms/step - loss: 0.0482 - acc: 0.9302 - val_loss: 0.0505 - val_acc: 0.9351
Epoch 26/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0422 - acc: 0.9469 - val_loss: 0.0462 - val_acc: 0.9221
Epoch 27/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0455 - acc: 0.9358 - val_loss: 0.0564 - val_acc: 0.9156
Epoch 28/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0450 - acc: 0.9385 - val_loss: 0.0542 - val_acc: 0.9156
Epoch 29/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0562 - acc: 0.9274 - val_loss: 0.0487 - val_acc: 0.9286
Epoch 30/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0473 - acc: 0.9385 - val_loss: 0.0634 - val_acc: 0.9156
Epoch 31/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0498 - acc: 0.9413 - val_loss: 0.0450 - val_acc: 0.9286
Epoch 32/50
36/36 [==============================] - 0s 2ms/step - loss: 0.0451 - acc: 0.9358 -